In [1]:
import pandas as pd
base_dir = 'data/McGill-Billboard'
data_index = 'billboard-2.0-manychords.csv'

df_songs = pd.read_csv(f'{base_dir}/{data_index}')
#df_songs = pd.read_csv(f'{data_index}') # colab
df_songs.set_index('id', inplace=True)
len(df_songs)

719

In [2]:
df_songs.head(n=3)

,title,artist,no_chord_percent
id,,,
3,I Don't Mind,James Brown,0.049747
4,You've Got A Friend,"Roberta Flack,Donny Hathaway",0.050770
6,The Rose,Bette Midler,0.117244


In [3]:
test_ids = [1289, 736, 637, 270, 18] # songs to exclude for testing
df_dataset = df_songs.drop(index=test_ids)
len(df_dataset)

714

#### Splitting

In [4]:
import numpy as np
_SEED = 0

df_idxs = np.array(df_dataset.index.values)
rng = np.random.default_rng(_SEED)
rng.shuffle(df_idxs)

df_idxs[:10]

array([1167,    6,  986,  227,  743,  568,  107,  181,   27,  793])

In [5]:
from dataloader import ChromaSequenceDataset
# import dataloader
# from importlib import reload
# reload(dataloader)

_LABEL_TYPE = 'majmin'
_SEQ_LEN = 64

pre_computed_seq = f'data/chordseq/{_LABEL_TYPE}_{_SEQ_LEN}.pkl'
#pre_computed_seq = f'{_LABEL_TYPE}_{_SEQ_LEN}.pkl' # colab
ds = ChromaSequenceDataset(pre_computed_sequence=pre_computed_seq)

Loaded sequence data.


In [6]:
for train_split, val_split in ds.get_next_cv_split(df_idxs):
    print(train_split.shape, val_split.shape)

((43541, 64, 24), (43541, 64)) ((6882, 64, 24), (6882, 64))
((43575, 64, 24), (43575, 64)) ((6848, 64, 24), (6848, 64))
((43448, 64, 24), (43448, 64)) ((6975, 64, 24), (6975, 64))
((43485, 64, 24), (43485, 64)) ((6938, 64, 24), (6938, 64))
((42843, 64, 24), (42843, 64)) ((7580, 64, 24), (7580, 64))


#### Evaluation

In [12]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = ""

from tensorflow import keras

_CKPT_PATH = 'models/chroma-seq-bilstm-crf-0'
model = keras.models.load_model(_CKPT_PATH)

In [13]:
type(model)

tensorflow.python.keras.saving.saved_model.load.ModelWithCRFLoss

In [21]:
from sklearn.metrics import accuracy_score
_BATCH_SIZE = 128

for train_split, val_split in ds.get_next_cv_split(df_idxs):
    preds, _, _, _ = model.predict(val_split.feats, batch_size=_BATCH_SIZE)
    acc = accuracy_score(val_split.labels.flatten(), preds.flatten())    
    break

In [22]:
print(acc) # RETRAIN!!

0.6496090344376635
